# Luftdaten data : data cleaning, resampling - mini version - cleaning v4

# - TRYING TO HEAD FOR FINAL… 
## Code builds a continuous time tabular version of the luftdaen data, such that the same time period is present for each sensor in the data, regardless of whether each sensor has data for all the time slots. 


#### Reference documents

Resampling time series data with Pandas ( Ben Alex Keen ) 
http://benalexkeen.com/resampling-time-series-data-with-pandas/

Pandas reference manual : 

.at - access df values using nay kind of index, for SINGLE VALUES
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iat - only integer index values for getting/setting SINGLE df values
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iat.html

.loc - 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc

.iloc - purely integer indexed access ( getting/setting ) values 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc

datetime - documentation - useful for time! 
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

rounding numbers in pyhton
https://realpython.com/python-rounding/

#### methods of filling … 

These are some of the common methods you might use for resampling:

Method	Description

bfill	Backward fill

count	Count of values

ffill	Forward fill

first	First valid data value

last	Last valid data value

max	Maximum data value

mean	Mean of values in time range

median	Median of values in time range

min	Minimum data value

nunique	Number of unique values

ohlc	Opening value, highest value, lowest value, closing value

pad	Same as forward fill

std	Standard deviation of values

sum	Sum of values

var	Variance of values

#### time abbreviations 

Alias	Description

B	Business day

D	Calendar day

W	Weekly

M	Month end

Q	Quarter end

A	Year end

BA	Business year end

AS	Year start

H	Hourly frequency

T, min	Minutely frequency

S	Secondly frequency

L, ms	Millisecond frequency

U, us	Microsecond frequency

N, ns	Nanosecond frequency

In [55]:
import pandas as pd
import numpy as np
import time

In [56]:
# parameters

# start_time = "2018-12-31 21:58:42"
end_time = "2019-01-01 11:58:42"
# generate this please
start_time = "?????"

time_frequency_for_periods__for_basic_data = "5T"
num_of_time_periods___for_basic_data = 24*20 # 24 hrs * 12 x 5 mins in each hour

# when generating time periods 
sampling_frequency = "3T"



# --- data urls 

curr_url = "????"
nordic_midnight_24_hrs_data__url = "/Users/miska/Documents/open_something/luftdaten/luftdaten_code/luftdaten__make_tabular_data__from_db_data/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"
# nordic_midnight_24_hrs_data__url = "/home/miska/documents/opensomething/luftdaten/dustmin_to_csv__various_code/ld_NYE_midnight_24hrs_nordics_all_data_01.csv"



# set the current data source 
curr_url = nordic_midnight_24_hrs_data__url

In [57]:
# try convert the timestamp in the data, to epoch

in_data = pd.read_csv( curr_url )
in_data.shape

(127109, 7)

#### basic variables ( set up )

In [58]:
# time length of sample period
time_length_of_sample_period__in_seconds = 60*3

# how many time sample periods can fit in the current total data time period
num_of_sample_time_periods_fit_in_total_sampled_period = 0 

In [59]:
## general operations - what kind of time length in the data, are we outputting

# for when we're creating 24 hour time segements
default__generate_data_for_24_hour_period_starting_from_starttime = 'default__generate_data_for_24_hour_period_starting_from_starttime'

# for when we're creating data files for the period until now
default__generate_data_from_last_midnight_until_current_time =  "default__generate_data_from_last_midnight_until_current_time"

# which are we using? 
current_time_duration_in_data_generation = default__generate_data_from_last_midnight_until_current_time

In [60]:
def figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration( which_time_duration_are_we_using, time_length_of_sample_period__in_seconds ):
    
    print(">>> figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration : checking time period : |"+which_time_duration_are_we_using+"| single sample time length : "+str(time_length_of_sample_period__in_seconds) )
    
    num_of_sample_time_periods_fit_in_total_sampled_period = 0
    
    if which_time_duration_are_we_using == 'default__generate_data_for_24_hour_period_starting_from_starttime':
        num_of_sample_time_periods_fit_in_total_sampled_period = int( ( 24*60*60 / time_length_of_sample_period__in_seconds ) ) 
    
    if which_time_duration_are_we_using == 'default__generate_data_from_last_midnight_until_current_time':
        ## - get time since midnight 
        
        # generate timestamp, to get time since midnight 
        end_timestamp = pd.Timestamp.now()
        hours_seconds_since_midnight = end_timestamp.hour*60*60
        min_seconds_since_midnight = end_timestamp.minute*60
        seconds_since_midnight = hours_seconds_since_midnight + min_seconds_since_midnight + end_timestamp.second
        
        print( "\n -- the number of seconds since midnight is "+str( seconds_since_midnight ) )
        
        num_of_sample_time_periods_fit_in_total_sampled_period = int( seconds_since_midnight / time_length_of_sample_period__in_seconds ) 
        
        
    print("\n - - - - FINALLY : num_of_sample_time_periods_fit_in_total_sampled_period : "+str( num_of_sample_time_periods_fit_in_total_sampled_period ))
    
    return num_of_sample_time_periods_fit_in_total_sampled_period

In [61]:
# test fetching the number of sample periods in 24 hours 
figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration( default__generate_data_for_24_hour_period_starting_from_starttime, time_length_of_sample_period__in_seconds )

>>> figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration : checking time period : |default__generate_data_for_24_hour_period_starting_from_starttime| single sample time length : 180

 - - - - FINALLY : num_of_sample_time_periods_fit_in_total_sampled_period : 480


480

In [62]:
# test fetching the number of sample periods since last midnight 
figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration( default__generate_data_from_last_midnight_until_current_time, time_length_of_sample_period__in_seconds )

>>> figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration : checking time period : |default__generate_data_from_last_midnight_until_current_time| single sample time length : 180

 -- the number of seconds since midnight is 34571

 - - - - FINALLY : num_of_sample_time_periods_fit_in_total_sampled_period : 192


192

In [63]:
int( ( 12*60*60 + 47*60 )/ time_length_of_sample_period__in_seconds ) 

255

In [64]:
figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration( default__generate_data_for_24_hour_period_starting_from_starttime, time_length_of_sample_period__in_seconds )

>>> figure_out_how_many_sample_time_periods_fit_in_desired_sample_time_duration : checking time period : |default__generate_data_for_24_hour_period_starting_from_starttime| single sample time length : 180

 - - - - FINALLY : num_of_sample_time_periods_fit_in_total_sampled_period : 480


480

#### Generate relevant timestamps

In [65]:
# timestamp last midnight 
end_timestamp = pd.Timestamp.now()
timestamp_last_midnight = pd.Timestamp( end_timestamp.year, end_timestamp.month, end_timestamp.day, 0 )

timestamp_last_midnight

Timestamp('2019-02-20 00:00:00')

In [66]:
timestamp_now = pd.Timestamp.now()
timestamp_now

Timestamp('2019-02-20 09:36:12.889313')

#### set up data 

In [67]:
in_data.dtypes

sensor_id         int64
sensor_namee     object
lat             float64
lon             float64
timestamp        object
p1              float64
p2              float64
dtype: object

In [68]:
# aha - timestamp column not a timestamp column?
# - let's fix 
in_data['timestamp'] = pd.to_datetime( in_data['timestamp'] )

In [69]:
# check the timestamps column type again
type( in_data['timestamp'][0] )

pandas._libs.tslibs.timestamps.Timestamp

In [70]:
# set the timestamp column as the index 
in_data = in_data.set_index( 'timestamp' )

In [71]:
in_data.index

DatetimeIndex(['2018-12-31 11:57:22', '2018-12-31 11:58:44',
               '2018-12-31 11:58:47', '2018-12-31 11:56:41',
               '2018-12-31 11:57:42', '2018-12-31 11:57:52',
               '2018-12-31 11:58:51', '2018-12-31 11:58:28',
               '2018-12-31 11:57:18', '2018-12-31 11:57:22',
               ...
               '2019-01-01 11:59:41', '2019-01-01 11:59:46',
               '2019-01-01 11:57:19', '2019-01-01 11:59:59',
               '2019-01-01 11:56:55', '2019-01-01 11:58:57',
               '2019-01-01 11:59:36', '2019-01-01 11:59:41',
               '2019-01-01 11:57:12', '2019-01-01 11:58:42'],
              dtype='datetime64[ns]', name='timestamp', length=127109, freq=None)

In [72]:
in_data = in_data.sort_index()

In [73]:
# check
in_data.index

DatetimeIndex(['2018-12-31 11:55:19', '2018-12-31 11:56:37',
               '2018-12-31 11:56:38', '2018-12-31 11:56:39',
               '2018-12-31 11:56:40', '2018-12-31 11:56:40',
               '2018-12-31 11:56:40', '2018-12-31 11:56:41',
               '2018-12-31 11:56:42', '2018-12-31 11:56:43',
               ...
               '2019-01-01 23:59:54', '2019-01-01 23:59:54',
               '2019-01-01 23:59:55', '2019-01-01 23:59:55',
               '2019-01-01 23:59:56', '2019-01-01 23:59:56',
               '2019-01-01 23:59:56', '2019-01-01 23:59:56',
               '2019-01-01 23:59:57', '2019-01-01 23:59:58'],
              dtype='datetime64[ns]', name='timestamp', length=127109, freq=None)

In [74]:
#### set up blank start and end rows

#### more data testing

In [75]:
in_data__START_TIME__blank_row = pd.DataFrame( data={"p1": np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [timestamp_last_midnight] ) )
in_data__START_TIME__blank_row

,p1,p2
2019-02-20,NaN,NaN


In [76]:
in_data__END_TIME__blank_row = pd.DataFrame( data={"p1": np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [end_timestamp] ) )
in_data__START_TIME__blank_row

,p1,p2
2019-02-20,NaN,NaN


In [77]:
test_data_df.columns

Index(['p1', 'p2'], dtype='object')

In [78]:
in_data_minimal = in_data[ ['p1', 'p2'] ]

In [79]:
in_data_minimal

,p1,p2
timestamp,,
2018-12-31 11:55:19,6.30,2.60
2018-12-31 11:56:37,174.80,15.13
2018-12-31 11:56:38,1.62,1.02
2018-12-31 11:56:39,33.95,13.40
2018-12-31 11:56:40,63.25,10.33
2018-12-31 11:56:40,11.45,6.00
2018-12-31 11:56:40,13.68,3.12
2018-12-31 11:56:41,11.05,6.62
2018-12-31 11:56:42,1.66,1.10


#### TEST insert the data

In [80]:
test_data_df__with__in_data_minimal = test_data_df.append( in_data_minimal, sort=False )

In [81]:
test_data_df__with__in_data_minimal.head()

,p1,p2
2019-02-20 09:19:44.561089,NaN,NaN
2018-12-31 11:55:19.000000,6.30,2.60
2018-12-31 11:56:37.000000,174.80,15.13
2018-12-31 11:56:38.000000,1.62,1.02
2018-12-31 11:56:39.000000,33.95,13.40


In [82]:
test_data_df__with__in_data_minimal__and_end_time = test_data_df__with__in_data_minimal.append( in_data__END_TIME__blank_row )

In [83]:
test_data_df__with__in_data_minimal__and_end_time.head(), test_data_df__with__in_data_minimal__and_end_time.tail()

(                                p1     p2
 2019-02-20 09:19:44.561089     NaN    NaN
 2018-12-31 11:55:19.000000    6.30   2.60
 2018-12-31 11:56:37.000000  174.80  15.13
 2018-12-31 11:56:38.000000    1.62   1.02
 2018-12-31 11:56:39.000000   33.95  13.40,
                               p1    p2
 2019-01-01 23:59:56.000000  1.85  0.80
 2019-01-01 23:59:56.000000  1.48  0.52
 2019-01-01 23:59:57.000000  7.26  1.60
 2019-01-01 23:59:58.000000  2.65  2.01
 2019-02-20 09:36:12.681632   NaN   NaN)